# Visualize pymoo result
Load the pymoo result object saved with pickle and visualize it.

ONOSControllerPlacement needs to be definded to load the result object.
So, execute the following cell first.

In [1]:
#!/usr/bin/env python
import numpy as np
import networkx as nx
import math
import pickle
from pymoo.core.problem import ElementwiseProblem

class ONOSControllerPlacement(ElementwiseProblem):
    def __init__(self, num_nodes, distance_matrix, shortest_paths, graph, **kwargs):
        super().__init__(n_var=2*num_nodes, 
                         n_obj=4, 
                         n_constr=2, 
                         xl=0, xu=1, 
                         **kwargs)
        self.num_nodes = num_nodes
        self.distance_matrix = distance_matrix
        self.shortest_paths = shortest_paths
        self.graph = graph
    
    def _evaluate(self, x, out, *args, **kwargs):
        controller_nodes = x[:self.num_nodes]   # first half is controller placement
        atomix_nodes = x[self.num_nodes:]       # second half is atomix placement


        num_controller = np.sum(controller_nodes)
        num_atomix = np.sum(atomix_nodes)

        # Obj1: Minimize number of contrtoller
        f1 = num_controller

        # Obj2: Minimize number of atomix
        f2 = num_atomix

        # Obj3: Minimize average FSP
        f3 = calculate_FST(self.num_nodes, 
                           controller_nodes, 
                           atomix_nodes, 
                           self.distance_matrix, 
                           self.shortest_paths)
        
        f4 = calculate_BC(self.num_nodes, 
                           controller_nodes, 
                           atomix_nodes, 
                           self.distance_matrix, 
                        #    self.shortest_paths,
                           self.graph)

        # Constr1: The number of controller is equal to or greater than 2
        g1 = 2 - num_controller

        # Constr2: The number of atomix is equal to or greater than 3
        g2 = 3 - num_atomix
        
        # Add the centrality metrix into optimazing objectives:
        # 1. Nearest controller for each switch
        # 2. The number of controlled switches for each controller should be <= limit_num_switches_controlled (limit_num_switches_controlled=int(math.ceil(num_nodes/num_controller)))
        # 3. return value should be the variance for all controller's betweenness centrality
        out["F"] = [f1, f2, f3, f4]
        out["G"] = [g1, g2]


def calculate_FST(num_nodes, controller_nodes, atomix_nodes, distance_matrix, shortest_paths):
    num_controller = np.sum(controller_nodes)
    num_atomix = np.sum(atomix_nodes)
    controller_list = np.nonzero(controller_nodes)[0].tolist()
    atomix_list = np.nonzero(atomix_nodes)[0].tolist()

    if(num_controller == 0 or num_atomix ==0):
        return math.inf

    # find the nearest controller for each switch
    controller_of = []
    for s in range(num_nodes):
        delay = math.inf
        nearest_controller = None
        for c in controller_list:
            if distance_matrix[s][c] < delay:
                delay = distance_matrix[s][c]
                nearest_controller = c
        controller_of.append(nearest_controller)    

    # calculate average delay to atomix nodes from each controller
    average_atomix_delay_from = {}
    for c in controller_list:
        delay = []
        for a in atomix_list:
            delay.append(distance_matrix[c][a])
        average_atomix_delay_from[c] = np.mean(delay)

    # find the nearest atomix for each atomix and calculate average delay
    atomix_atomix_delays = []
    for a1 in atomix_list:
        delay = math.inf
        for a2 in atomix_list:
            if(a1 == a2):
                continue
            if distance_matrix[a1][a2] < delay:
                delay = distance_matrix[a1][a2]
        atomix_atomix_delays.append(delay)
    average_atomix_atomix_delay = np.mean(atomix_atomix_delays)
    FTSs = []
    for source in range(num_nodes):
        for distination in range(num_nodes):
            if(source == distination):
                continue
            delay = 0
            is_controlled_by_single_controller = True
            counted_controllers = []
            for s in shortest_paths[source][distination]:
                # switch-controller delay
                delay += distance_matrix[s][controller_of[s]] * 4

                # controller-atomix delay
                if(s == source):
                    delay += average_atomix_delay_from[controller_of[s]] * 2
                elif(s != distination):
                    if(controller_of[s] != controller_of[source]):
                        is_controlled_by_single_controller = False
                        if(not controller_of[s] in counted_controllers):
                            counted_controllers.append(controller_of[s])
                            delay += average_atomix_delay_from[controller_of[s]]
                else:
                    if(controller_of[s] == controller_of[source]):
                        if(not is_controlled_by_single_controller):
                            delay += average_atomix_delay_from[controller_of[s]]
                    else:
                        delay += average_atomix_delay_from[controller_of[s]] * 2
            
            # atomix-atomix delay
            delay +=  average_atomix_atomix_delay * 2
            FTSs.append(delay)

    return np.mean(FTSs)



def calculate_BC(num_nodes, controller_nodes, atomix_nodes, distance_matrix, graph):
    G = nx.Graph()
    for node1 in range(len(graph)):
        G.add_node(str(node1))
        for node2, delay in graph[node1].items():
            G.add_edge(str(node1), str(node2), weight=delay)
    
    # The list of betweenness centrality for all switches
    nodes_bc=nx.current_flow_betweenness_centrality(G, normalized=True, weight=None, dtype='float', solver='full')
    num_controller = np.sum(controller_nodes)
    num_atomix = np.sum(atomix_nodes)
    controller_list = np.nonzero(controller_nodes)[0].tolist()

    if(num_controller == 0 or num_atomix ==0):
        return math.inf

    # find the nearest controller for each switch
    controller_of = []
    limit_num_switches_controlled=int(math.ceil(num_nodes/num_controller)) # balance the number of switches controllers can control 
    switches_bc_of_controller_ = dict.fromkeys((range(num_nodes)),0) # list of sum of betweenness centrality of switches for each controller
    for s in range(num_nodes):
        delay = math.inf
        nearest_controller = None
        controlled_switches=[]
        for c in controller_list:
            # Conditions: nearest controller (with the lowest delay) && the number of switches for each controller < limit_num_switches_controlled
            if distance_matrix[s][c] < delay and controller_of.count(c) < limit_num_switches_controlled:
                delay = distance_matrix[s][c]
                nearest_controller = c
                controlled_switches.append(s)
        switches_bc_of_controller_[nearest_controller] += nodes_bc[str(s)]
        controller_of.append(nearest_controller)
    
    # Simplify switches_bc_of_controller_ (only need value for calculating variance)
    bc_array = []
    for i in switches_bc_of_controller_.values():
        bc_array.append(i)

    # return variance value can show the degree of balance within all controllers
    return np.var(bc_array)

### Load rnsga2 results for all topos

In [2]:
with open('res_bc_Cogent_rnsga2.pkl','rb') as f_Cogent:
    res_5_Cogent = pickle.load(f_Cogent)
with open('res_bc_UsCarrier_rnsga2.pkl','rb') as f_UsCarrier:
    res_5_UsCarrier = pickle.load(f_UsCarrier)
with open('res_bc_HiberniaGlobal_rnsga2.pkl','rb') as f_HiberniaGlobal:
    res_5_HiberniaGlobal = pickle.load(f_HiberniaGlobal)
with open('res_bc_Colt_rnsga2.pkl','rb') as f_Colt:
    res_5_Colt = pickle.load(f_Colt)
with open('res_bc_Funet_rnsga2.pkl','rb') as f_Funet:
    res_5_Funet = pickle.load(f_Funet)
with open('res_bc_Abvt_rnsga2.pkl','rb') as f_Abvt:
    res_5_Abvt = pickle.load(f_Abvt)
with open('res_bc_Intellifiber_rnsga2.pkl','rb') as f_Intellifiber:
    res_5_Intellifiber = pickle.load(f_Intellifiber)
with open('res_bc_TataNld_rnsga2.pkl','rb') as f_TataNld:
    res_5_TataNld = pickle.load(f_TataNld)
with open('res_bc_Internode_rnsga2.pkl','rb') as f_Internode:
    res_5_Internode = pickle.load(f_Internode)
with open('res_bc_Missouri_rnsga2.pkl','rb') as f_Missouri:
    res_5_Missouri = pickle.load(f_Missouri)
with open('res_bc_Ion_rnsga2.pkl','rb') as f_Ion:
    res_5_Ion = pickle.load(f_Ion)
with open('res_bc_Palmetto_rnsga2.pkl','rb') as f_Palmetto:
    res_5_Palmetto = pickle.load(f_Palmetto)

## Hypervolume
1. Store values

In [ ]:
# rnsga2
F5_Cogent=res_5_Cogent.F
F5_UsCarrier=res_5_UsCarrier.F
F5_HiberniaGlobal=res_5_HiberniaGlobal.F
F5_Colt=res_5_Colt.F
F5_Funet=res_5_Funet.F
F5_Abvt=res_5_Abvt.F
F5_Intellifiber=res_5_Intellifiber.F
F5_TataNld=res_5_TataNld.F
F5_Internode=res_5_Internode.F
F5_Missouri=res_5_Missouri.F
F5_Ion=res_5_Ion.F
F5_Palmetto=res_5_Palmetto.F

# # Nadir Point from 9 Algorithms: 
# ref_point = [1.81000000e+02 6.20000000e+01 3.92281875e+03 1.70537209e+00]
# # Nadir Point from 4 Algorithms: 
# ref_point = [160.  77.  inf  inf]
# # Nadir Point from all (13) Algorithms: 
# ref_point = [181.  77.  inf  inf]

ref_point = [181, 77, 3922.81875, 1.70537209]

hist_F5_Cogent = []
hist_F5_UsCarrier = []
hist_F5_HiberniaGlobal = []
hist_F5_Colt = []
hist_F5_Funet = []
hist_F5_Abvt = []
hist_F5_Intellifiber = []
hist_F5_TataNld = []
hist_F5_Internode = []
hist_F5_Missouri = []
hist_F5_Ion = []
hist_F5_Palmetto = []


for algo5_Cogent in res_5_Cogent.history:
    opt5_Cogent = algo5_Cogent.opt
    feas5_Cogent = np.where(opt5_Cogent.get("feasible"))[0]
    hist_F5_Cogent.append(opt5_Cogent.get("F")[feas5_Cogent])
for algo5_UsCarrier in res_5_UsCarrier.history:
    opt5_UsCarrier = algo5_UsCarrier.opt
    feas5_UsCarrier = np.where(opt5_UsCarrier.get("feasible"))[0]
    hist_F5_UsCarrier.append(opt5_UsCarrier.get("F")[feas5_UsCarrier])
for algo5_HiberniaGlobal in res_5_HiberniaGlobal.history:
    opt5_HiberniaGlobal = algo5_HiberniaGlobal.opt
    feas5_HiberniaGlobal = np.where(opt5_HiberniaGlobal.get("feasible"))[0]
    hist_F5_HiberniaGlobal.append(opt5_HiberniaGlobal.get("F")[feas5_HiberniaGlobal])
for algo5_Colt in res_5_Colt.history:
    opt5_Colt = algo5_Colt.opt
    feas5_Colt = np.where(opt5_Colt.get("feasible"))[0]
    hist_F5_Colt.append(opt5_Colt.get("F")[feas5_Colt])
for algo5_Funet in res_5_Funet.history:
    opt5_Funet = algo5_Funet.opt
    feas5_Funet = np.where(opt5_Funet.get("feasible"))[0]
    hist_F5_Funet.append(opt5_Funet.get("F")[feas5_Funet])
for algo5_Abvt in res_5_Abvt.history:
    opt5_Abvt = algo5_Abvt.opt
    feas5_Abvt = np.where(opt5_Abvt.get("feasible"))[0]
    hist_F5_Abvt.append(opt5_Abvt.get("F")[feas5_Abvt])
for algo5_Intellifiber in res_5_Intellifiber.history:
    opt5_Intellifiber = algo5_Intellifiber.opt
    feas5_Intellifiber = np.where(opt5_Intellifiber.get("feasible"))[0]
    hist_F5_Intellifiber.append(opt5_Intellifiber.get("F")[feas5_Intellifiber])
for algo5_TataNld in res_5_TataNld.history:
    opt5_TataNld = algo5_TataNld.opt
    feas5_TataNld = np.where(opt5_TataNld.get("feasible"))[0]
    hist_F5_TataNld.append(opt5_TataNld.get("F")[feas5_TataNld])
for algo5_Internode in res_5_Internode.history:
    opt5_Internode = algo5_Internode.opt
    feas5_Internode= np.where(opt5_Internode.get("feasible"))[0]
    hist_F5_Internode.append(opt5_Internode.get("F")[feas5_Internode])
for algo5_Missouri in res_5_Missouri.history:
    opt5_Missouri = algo5_Missouri.opt
    feas5_Missouri = np.where(opt5_Missouri.get("feasible"))[0]
    hist_F5_Missouri.append(opt5_Missouri.get("F")[feas5_Missouri])
for algo5_Ion in res_5_Ion.history:
    opt5_Ion = algo5_Ion.opt
    feas5_Ion = np.where(opt5_Ion.get("feasible"))[0]
    hist_F5_Ion.append(opt5_Ion.get("F")[feas5_Ion])
for algo5_Palmetto in res_5_Palmetto.history:
    opt5_Palmetto = algo5_Palmetto.opt
    feas5_Palmetto = np.where(opt5_Palmetto.get("feasible"))[0]
    hist_F5_Palmetto.append(opt5_Palmetto.get("F")[feas5_Palmetto])

2. Calculate hypervolume and merge them (Mean value) for rnsga2 algorithm

In [ ]:
from pymoo.indicators.hv import Hypervolume

metric = Hypervolume(ref_point= np.array(ref_point))

hv_5_Cogent = [metric.do(_F5_Cogent) for _F5_Cogent in hist_F5_Cogent]
hv_5_UsCarrier = [metric.do(_F5_UsCarrier) for _F5_UsCarrier in hist_F5_UsCarrier]
hv_5_HiberniaGlobal = [metric.do(_F5_HiberniaGlobal) for _F5_HiberniaGlobal in hist_F5_HiberniaGlobal]
hv_5_Colt = [metric.do(_F5_Colt) for _F5_Colt in hist_F5_Colt]
hv_5_Funet = [metric.do(_F5_Funet) for _F5_Funet in hist_F5_Funet]
hv_5_Abvt = [metric.do(_F5_Abvt) for _F5_Abvt in hist_F5_Abvt]
hv_5_Intellifiber = [metric.do(_F5_Intellifiber) for _F5_Intellifiber in hist_F5_Intellifiber]
hv_5_TataNld = [metric.do(_F5_TataNld) for _F5_TataNld in hist_F5_TataNld]
hv_5_Internode = [metric.do(_F5_Internode) for _F5_Internode in hist_F5_Internode]
hv_5_Missouri = [metric.do(_F5_Missouri) for _F5_Missouri in hist_F5_Missouri]
hv_5_Ion = [metric.do(_F5_Ion) for _F5_Ion in hist_F5_Ion]
hv_5_Palmetto = [metric.do(_F5_Palmetto) for _F5_Palmetto in hist_F5_Palmetto]

# rnsga2
hv_5 = [(hv_Cogent + hv_UsCarrier + hv_HiberniaGlobal + hv_Colt + hv_Funet + hv_Abvt + hv_Intellifiber + hv_TataNld + hv_Internode + hv_Missouri + hv_Ion + hv_Palmetto) / 12 for hv_Cogent, hv_UsCarrier, hv_HiberniaGlobal, hv_Colt, hv_Funet, hv_Abvt, hv_Intellifiber, hv_TataNld, hv_Internode, hv_Missouri, hv_Ion, hv_Palmetto in zip(hv_5_Cogent, hv_5_UsCarrier, hv_5_HiberniaGlobal, hv_5_Colt, hv_5_Funet, hv_5_Abvt, hv_5_Intellifiber, hv_5_TataNld, hv_5_Internode, hv_5_Missouri, hv_5_Ion, hv_5_Palmetto)]

In [ ]:
print('The hypervolumn for Cogent:')
print(hv_5_Cogent)
print('The hypervolumn for UsCarrier:')
print(hv_5_UsCarrier)
print('The hypervolumn for HiberniaGlobal:')
print(hv_5_HiberniaGlobal)
print('The hypervolumn for Colt:')
print(hv_5_Colt)
print('The hypervolumn for Funet:')
print(hv_5_Funet)
print('The hypervolumn for Abvt:')
print(hv_5_Abvt)
print('The hypervolumn for Intellifiber:')
print(hv_5_Intellifiber)
print('The hypervolumn for TataNld:')
print(hv_5_TataNld)
print('The hypervolumn for Internode:')
print(hv_5_Internode)
print('The hypervolumn for Missouri:')
print(hv_5_Missouri)
print('The hypervolumn for Ion:')
print(hv_5_Ion)
print('The hypervolumn for Palmetto:')
print(hv_5_Palmetto)

# Show all hv for each topo
import matplotlib.pyplot as plt

plt.figure(figsize=(7, 5))
plt.plot(list(range(1, len(hv_5_Cogent)+1)), hv_5_Cogent,  color='black', label='Cogent')
plt.plot(list(range(1, len(hv_5_UsCarrier)+1)), hv_5_UsCarrier,  color='blue', label='UsCarrier')
plt.plot(list(range(1, len(hv_5_HiberniaGlobal)+1)), hv_5_HiberniaGlobal,  color='red', label='HiberniaGlobal')
plt.plot(list(range(1, len(hv_5_Colt)+1)), hv_5_Colt,  color='orange', label='Colt')
plt.plot(list(range(1, len(hv_5_Funet)+1)), hv_5_Funet,  color=(0, 0, 1, 0.5), label='Funet')
plt.plot(list(range(1, len(hv_5_Abvt)+1)), hv_5_Abvt,  color='gray', label='Abvt')
plt.plot(list(range(1, len(hv_5_Intellifiber)+1)), hv_5_Intellifiber,  color='purple', label='Intellifiber')
plt.plot(list(range(1, len(hv_5_TataNld)+1)), hv_5_TataNld,  color='green', label='TataNld')
plt.plot(list(range(1, len(hv_5_Internode)+1)), hv_5_Internode,  color='#33FF57', label='Internode')
plt.plot(list(range(1, len(hv_5_Missouri)+1)), hv_5_Missouri,  color='brown', label='Missouri')
plt.plot(list(range(1, len(hv_5_Ion)+1)), hv_5_Ion,  color='pink', label='Ion')
plt.plot(list(range(1, len(hv_5_Palmetto)+1)), hv_5_Palmetto,  color='olive', label='Palmetto')
plt.title("Convergence")
plt.xlabel("Topos")
plt.ylabel("Hypervolume")
plt.legend()
plt.show()

In [ ]:
print('The aveage of hypervolumn for rnsga2:')
print(hv_5)

3. Draw mean hypervolume for rnsga2

In [ ]:
import matplotlib.pyplot as plt

#  Show all alogrithms' hv into a single figure
plt.figure(figsize=(7, 5))
# plt.plot(list(range(1, len(hv_1)+1)), hv_1,  color='black', label='AGEMOEA')
# plt.plot(list(range(1, len(hv_2)+1)), hv_2,  color='blue', label='AGEMOEA2')
# plt.plot(list(range(1, len(hv_3)+1)), hv_3,  color='red', label='NSGA2')
# plt.plot(list(range(1, len(hv_4)+1)), hv_4,  color='orange', label='NSGA3')
plt.plot(list(range(1, len(hv_5)+1)), hv_5,  color=(0, 0, 1, 0.5), label='RNSGA2')
# plt.plot(list(range(1, len(hv_6)+1)), hv_6,  color='gray', label='RNSGA3')
# plt.plot(list(range(1, len(hv_7)+1)), hv_7,  color='purple', label='RVEA')
# plt.plot(list(range(1, len(hv_8)+1)), hv_8,  color='green', label='SMSEMOA')
# plt.plot(list(range(1, len(hv_9)+1)), hv_9,  color='#33FF57', label='UNSGA3')
# plt.plot(list(range(1, len(hv_10)+1)), hv_10,  color='brown', label='CTAEA')
# plt.plot(list(range(1, len(hv_11)+1)), hv_11,  color='pink', label='DNSGA2')
# plt.plot(list(range(1, len(hv_12)+1)), hv_12,  color='olive', label='KGBDMOEA')
# plt.plot(list(range(1, len(hv_13)+1)), hv_13,  color='cyan', label='MOEAD')
plt.title("Convergence")
plt.xlabel("Generations") # brown,pink,#5733FF,olive,cyan,#FF5733,#33FF57
plt.ylabel("Hypervolume")
plt.legend()
plt.show()